In [0]:
df2 = spark.read.json('/Volumes/databricks_simulated_retail_customer_data/v01/retail-pipeline/orders/stream_json/00.json')

display(df2.limit(5))
df2.printSchema()

In [0]:
df2.count()

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

order_stream_schema = StructType([
    StructField("customer_id", IntegerType()),
    StructField("notifications", StringType()),
    StructField("order_id", IntegerType()),
    StructField("order_timestamp", LongType())
])

In [0]:
df = spark.readStream.format("cloudFiles") \
  .option("cloudFiles.format", "json") \
  .option("cloudFiles.schemaLocation" , '/Volumes/analytics_star_schema/deltashare/checkpointing/orders/schema')\
    .schema(order_stream_schema)\
    .load('/Volumes/databricks_simulated_retail_customer_data/v01/retail-pipeline/orders/stream_json/')\



In [0]:
df = df.withColumn('order_timestamp', date_format(from_unixtime(col('order_timestamp')), 'yyyy-MM-dd'))\
       .withColumn('customer_id', col('customer_id').cast('integer'))\
       .withColumn('file_name' , col("_metadata.file_path"))
    



In [0]:
df.writeStream.format('delta')\
        .outputMode('append')\
        .trigger(once=True)\
        .option('checkpointLocation', '/Volumes/analytics_star_schema/deltashare/checkpointing/cp/orders')\
        .table('analytics_star_schema.deltashare.orders_stream')

In [0]:
%sql
select * from analytics_star_schema.deltashare.orders_stream limit 5

In [0]:
df = spark.read.table('analytics_star_schema.deltashare.orders_stream')

df.count()

In [0]:
%sql
select file_name,count(*) as row_count from analytics_star_schema.deltashare.orders_stream 
group by file_name
order by row_count desc

In [0]:
display(df.limit(5))